<a href="https://colab.research.google.com/github/trefftzc/cis677/blob/main/Julia_Colab_Notebook_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <img src="https://github.com/JuliaLang/julia-logo-graphics/raw/master/images/julia-logo-color.png" height="100" /> _Colab Notebook Template_

## Instructions
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. If you need a GPU: _Runtime_ > _Change runtime type_ > _Harware accelerator_ = _GPU_.
3. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia and other packages (if needed, update `JULIA_VERSION` and the other parameters). This takes a couple of minutes.
4. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the next section.

_Notes_:
* If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2, 3 and 4.
* After installation, if you want to change the Julia version or activate/deactivate the GPU, you will need to reset the Runtime: _Runtime_ > _Factory reset runtime_ and repeat steps 3 and 4.

In [1]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Unrecognized magic `%%shell`.

Julia does not use the IPython `%magic` syntax.   To interact with the IJulia kernel, use `IJulia.somefunction(...)`, for example.  Julia macros, string macros, and functions can be used to accomplish most of the other functionalities of IPython magics.


# Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system:

In [2]:
versioninfo()

Julia Version 1.10.9
Commit 5595d20a287 (2025-03-10 12:51 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 2 × Intel(R) Xeon(R) CPU @ 2.00GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-15.0.7 (ORCJIT, skylake-avx512)
Threads: 2 default, 0 interactive, 1 GC (on 2 virtual cores)
Environment:
  LD_LIBRARY_PATH = /usr/lib64-nvidia
  JULIA_NUM_THREADS = auto


There is a nice pdf file with a tutorial about Julia online. It is available at: https://www.sas.upenn.edu/~jesusfv/Chapter_HPC_8_Julia.pdf


# Packages
A package is a code that extends the basic capabilities Julia
with additional functions, data structures, etc. In such a way, Julia follows the modern
trend of open source software of having a base installation of a project and a lively ecosystem
of developers creating specialized tools that you can add or remove at will.

One of the first things you may want to do after installing
Julia is to add some useful packages. Recall that the first thing you need is to switch to the
package manager mode with ] .

You can check the packages that are currently installed with: **st**

You can add a package with: **add NameOfThePackage**

You can update a package with: **up NameOfThePackage**

You can remove a package with: **rm NameOfthePackage**

After a package has been installed, one use the statement:
** using NameOfThePackage **
to indicate that the code will use that package.

In the code cell below, the code installs the BenchmarkTools package,
and then uses it.

To benchmark a particular function, one use the\
** btime ** decorator, just before invoking a method.

In the code below, a matrix multiplication.

In [ ]:
import Pkg; Pkg.add("BenchmarkTools")
using BenchmarkTools

M = rand(2^11, 2^11)

@btime $M * $M;

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed BenchmarkTools ─ v1.6.0
    Updating `~/.julia/environments/v1.10/Project.toml`
  [6e4b80f9] + BenchmarkTools v1.6.0
    Updating `~/.julia/environments/v1.10/Manifest.toml`
  [6e4b80f9] + BenchmarkTools v1.6.0
  [9abbd945] + Profile
Precompiling packages...
   2752.7 ms  ✓ BenchmarkTools
  1 dependency successfully precompiled in 13 seconds. 460 already precompiled.


  546.234 ms (2 allocations: 32.00 MiB)


# Types

Julia has variables, values, and types. A variable is a name bound to a value. Julia is case
sensitive: a is a different variable than A . In fact, as we will see below, the variable can
be nearly any combination of Unicode characters. A value is a content (1, 3.2, ”economics”,
etc.). Technically, Julia considers that all values are objects (an object is an entity with
some attributes). This makes Julia closer to pure object-oriented languages such as Ruby
than to languages such as C++, where some values such as floating points are not objects.
Finally, values have types (i.e., integer, float, boolean, string, etc.). A variable does not have
a type, its value has. Types specify the attributes of the content. Functions in Julia will
look at the type of the values passed as operands and decide, according to them, how we
can operate on the values (i.e., which of the methods available to the function to apply).
Adding 1+2 (two integers) will be different than summing 1.0+2.0 (two floats) because the
method for summing two integers is different from the method to sum two floats. In the base
implementation of Julia, there are 230 different methods for the function sum! You can list
them with the command methods() as in:

In [3]:
methods(+)

# 189 methods for generic function "+" from Base:
   [1] +(t::Dates.Time, dt::Dates.Date)
     @ Dates /usr/local/share/julia/stdlib/v1.10/Dates/src/arithmetic.jl:22
   [2] +(x::Dates.Time, y::Dates.TimePeriod)
     @ Dates /usr/local/share/julia/stdlib/v1.10/Dates/src/arithmetic.jl:85
   [3] +(x::Bool, z::Complex{Bool})
     @ complex.jl:305
   [4] +(x::Bool, y::Bool)
     @ bool.jl:166
   [5] +(x::Bool)
     @ bool.jl:163
   [6] +(x::Bool, z::Complex)
     @ complex.jl:312
   [7] +(x::Real, z::Complex{Bool})
     @ complex.jl:319
   [8] +(x::Bool, y::T) where T<:AbstractFloat
     @ bool.jl:173
   [9] +(z::Complex{Bool}, x::Bool)
     @ complex.jl:306
  [10] +(z::Complex{Bool}, x::Real)
     @ complex.jl:320
  [11] +(z::Complex, x::Bool)
     @ complex.jl:313
  [12] +(x::BigInt, y::BigInt)
     @ Base.GMP gmp.jl:501
  [13] +(a::BigInt, b::BigInt, c::BigInt)
     @ Base.GMP gmp.jl:541
  [14] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt)
     @ Base.GMP gmp.jl:542
  [15] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt, e::BigInt)
     @ Base.GMP gmp.jl:543
  [16] +(x::BigInt, y::BigInt, rest::BigInt...)
     @ Base.GMP gmp.jl:683
  [17] +(c::BigInt, x::BigFloat)
     @ Base.MPFR mpfr.jl:468
  [18] +(x::BigInt, c::Union{UInt16, UInt32, UInt64, UInt8})
     @ Base.GMP gmp.jl:549
  [19] +(x::BigInt, c::Union{Int16, Int32, Int64, Int8})
     @ Base.GMP gmp.jl:555
  [20] +(x::Dates.CompoundPeriod, y::Dates.CompoundPeriod)
     @ Dates /usr/local/share/julia/stdlib/v1.10/Dates/src/periods.jl:334
  [21] +(x::Dates.CompoundPeriod, y::Dates.TimeType)
     @ Dates /usr/local/share/julia/stdlib/v1.10/Dates/src/periods.jl:362
  [22] +(x::Dates.CompoundPeriod, y::Dates.Period)
     @ Dates /usr/local/share/julia/stdlib/v1.10/Dates/src/periods.jl:332
  [23] +(B::BitMatrix, J::LinearAlgebra.UniformScaling)
     @ LinearAlgebra /usr/local/share/julia/stdlib/v1.10/LinearAlgebra/src/uniformscaling.jl:151
  [24] +(x::Dates.Date, y::Dates.Day)
     @ Dates /usr/local/share/julia/stdlib/v1.10/Dates/src/arithmetic.jl:81
  [25] +(x::Dates.Date, y::Dates.Week)
     @ Dates /usr/local/share/julia/stdlib/v1.10/Dates/src/arithmetic.jl:79
  [26] +(x::Dates.Date, y::Dates.Quarter)
     @ Dates /usr/local/share/julia/stdlib/v1.10/Dates/src/arithmetic.jl:75
  [27] +(dt::Dates.Date, z::Dates.Month)
     @ Dates /usr/local/share/julia/stdlib/v1.10/Dates/src/arithmetic.jl:56
  [28] +(dt::Dates.Date, y::Dates.Year)
     @ Dates /usr/local/share/julia/stdlib/v1.10/Dates/src/arithmetic.jl:29
  [29] +(dt::Dates.Date, t::Dates.Time)
     @ Dates /usr/local/share/julia/stdlib/v1.10/Dates/src/arithmetic.jl:21
  [30] +(::Missing, ::Missing)
     @ missing.jl:122
  [31] +(::Missing)
     @ missing.jl:101
  [32] +(x::Missing, y::Dates.AbstractTime)
     @ Dates /usr/local/share/julia/stdlib/v1.10/Dates/src/arithmetic.jl:91
  [33] +(::Missing, ::Number)
     @ missing.jl:123
  [34] +(x::Dates.DateTime, y::Dates.Quarter)
     @ Dates /usr/local/share/julia/stdlib/v1.10/Dates/src/arithmetic.jl:77
  [35] +(dt::Dates.DateTime, z::Dates.Month)
     @ Dates /usr/local/share/julia/stdlib/v1.10/Dates/src/arithmetic.jl:49
  [36] +(dt::Dates.DateTime, y::Dates.Year)
     @ Dates /usr/local/share/julia/stdlib/v1.10/Dates/src/arithmetic.jl:25
  [37] +(x::Dates.DateTime, y::Dates.Period)
     @ Dates /usr/local/share/julia/stdlib/v1.10/Dates/src/arithmetic.jl:83
  [38] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat, e::BigFloat)
     @ Base.MPFR mpfr.jl:619
  [39] +(x::BigFloat, y::BigFloat)
     @ Base.MPFR mpfr.jl:432
  [40] +(a::BigFloat, b::BigFloat, c::BigFloat)
     @ Base.MPFR mpfr.jl:606
  [41] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat)
     @ Base.MPFR mpfr.jl:612
  [42] +(x::BigFloat, c::BigInt)
     @ Base.MPFR mpfr.jl:463
  [43] +(x::BigFloat, c::Union{UInt16, UInt32, UInt64, UInt8})
     @ Base.MPFR mpfr.jl:439
  [44] +(x::BigFloat, c::Union{Int16, Int32, Int64, Int8})
     @ Base.MPFR mpfr.jl:447
  [45] +(x::BigFloat, c::Union{Float16, Float32, Float64})
 

# Polymorphic multiple dispatch

This application of different methods to a common function is known as polymorphic multiple
dispatch and it is one of the key concepts in Julia you need to understand.

Multiple dispatch is different from the overloading of operators existing in languages such as C++ because
it is determined at run time, not compilation time. Later, when we introduce composite types, we will see a
second difference: in Julia, methods are not defined within classes as you would do in most object-oriented
languages.

The previous paragraph may help to see why Julia is a strongly dynamically typed
programming language. Being a typed language means that the type of each value must be
known by the compiler at run time to decide which method to apply to that value. Being a
dynamically typed language means that such knowledge can be either explicit (i.e., declared
by the user) or implicit (i.e., deduced by Julia with an intelligent type inference engine from
the context it is used). Dynamic typing makes developing code with Julia flexible and fast:
you do not need to worry about explicitly type every value as you go along (i.e., declaring to
which type the value belongs). Being a strongly typed language means that you cannot use
a value of one type as another value, although you can convert it or let the compiler do it for
you. For example, Julia follows a promotion system where values of different types being
operated jointly are “promoted” to a common system: in the sum between an integer and a
float, the integer is “promoted” to float.10 You can, nevertheless, impose that the compiler
will not vary the type of a value to avoid subtle bugs in issues where the type is of critical
importance such as array indexing and, sometimes, to improve performance by providing
guidance to the JIT compiler on which methods to implement.

You do not need, though, to remember the type tree hierarchy, since Julia provides you
with commands to check the supertype (i.e., the type above a current type in the tree) and
subtype (i.e., the types below) of any given type:

In [2]:
supertype(Float64)

AbstractFloat

In [1]:

subtypes(Integer)

3-element Vector{Any}:
 Bool
 Signed
 Unsigned

You can always check the type of a variable with

In [4]:
a = 10
typeof(a)

Int64

## Variables

By default, integers values will be Int64 and floating point values will be
Float64 , but we also have shorter and longer types. Particularly
useful for computations with absolute large numbers (this happens sometimes, for example,
when evaluating likelihood functions), we have BigFloat. In the unlikely case that BigFloat
does not provide you with enough precission, Julia can use the GNU Multiple Precision
arithmetic (GMP) (https://gmplib.org/) and the GNU MPFR Libraries (http://www.
mpfr.org/).

In [8]:
a = 3 # integer
a = 0x3 # unsigned integer, hexadecimal base
a = 0b11 # unsigned integer, binary base
a = 3.0 # Float64
a = 4 + 3im # imaginary
a = complex(4,3) # same as above
a = true # boolean
a = "String" # string

"String"

# Minimum and Maximum for every type
You can check the minimum and maximum value every type can store with the functions
typemin() and typemax() , the machine precision of a type with eps() and, if it is a floating point, the effective bits in its mantissa by precision() . For example, for a
Float64 :

In [7]:
typemin(Float64) # returns -Inf (just a convention)


-Inf

In [8]:
typemax(Float64) # returns Inf (just a convention)


Inf

In [9]:
eps(Float64) # returns 2.22e-16


2.220446049250313e-16

In [10]:
precision(Float64) # returns 53

53

Larger or smaller numbers than the limits will return an overflow error. You can also check
the binary representation of a value:

In [10]:
a = 1
bitstring(a) # binary representation of a

"0000000000000000000000000000000000000000000000000000000000000001"

## Arithmetic Operators

Julia can handle all the common arithmetic operators:
+ - * / ^ # arithmetic operations

+. -. *. /. ^. # element-by-element operations (for vectors and matrices)

// # division for rationals that produces another rational

+a # identity operator

-a # negative of a

a+=1 # a = a+1, can be applied to any operator

a\b # b/a

div(a,b) # a/b, truncated to an integer

cld(a,b) # ceiling division

fld(a,b) # flooring division

rem(a,b) # remainder of a/b

mod(a,b) # module a,b

mod1(a,b) # module a,b after flooring division

gcd(a,b) # greatest positive common denominator of a,b

gcdx(a,b) # gcd of a and and and their minimal Bezout coefficients

lcm(a,b) # least common multiple of a,b

and some min-max operators

min(a,b) # min of a and (can take as many arguments as desired)

max(a,b) # max of a and (can take as many arguments as desired)

minmax(a,b) # min and max of a and b (a tuple return)

muladd(a,b,c) # a*b+c

Note, in particular, the use of the . to vectorize an operation (i.e., to apply an operation
to a vector or matrix instead of an scalar). While Julia does not require vectorized code to
achieve high performance (this is delivered through multiple dispatch and JIT compilation),
vectorized code is often easier to write, read, and debug.

Julia also accepts the alternative
notation
+(a,b)

# Logical Operators

Julia has all the widely-used logical operators:
! # not

&& # and

|| # or

== # is equal?

!== # is not equal?

=== # is equal? (enforcing type 2===2.0 is false

\> # bigger than

\>= # bigger or equal than

< # less than

<= # less or equal than

Logical operators can be linked with as much depth as desired:

3 > 2 && 4<=8 || 7 < 7.1

Note that the logical operators are lazy in Julia (in fact, all functions in Julia are lazy and
logical operators are just one example of functions). That is, they are only evaluated when
needed:

2 > 3 && println("I am lazy")

# Arrays

Julia makes arrays first-class components of the
language. An array is an ordered collection of objects stored in a multi-dimensional grid. For
example, an abstract 2 × 2 array of floats can be created with the simple constructor:

a = Array{Float64}(undef,2,2)



In [1]:
a = Array{Float64}(undef,2,2)

2×2 Matrix{Float64}:
 6.4758e-319   0.0
 6.78098e-310  0.0

Arrays can contain objects of any arbitrary type:

In [2]:
a = ["Economics" 2;
3.1 true]

2×2 Matrix{Any}:
  "Economics"     2
 3.1           true

Component a[1,1] is a string, component a[1,2] is an integer, component a[2,1] is a
float, and component a[2,1] is a boolean. Note that the access to an element of the array
is with square brackets [] , not with circular brackets () as in Matlab.

While Julia has specific Vector and Matrix types, these types are nothing but aliases
for one- and two-dimensional arrays (one dimensional arrays are also called flat arrays). Thus,
when no ambiguity occurs, and to facilitate explanation, we will refer to one-dimensional
arrays of numbers (integers, reals, complex) as vectors and to two-dimensional arrays of
numbers as matrices.

Note that Julia indexes arrays starting a 1, not at 0 as C/C++. For scientific computations Julia’s
convention is the only sensible approach.

A fundamental property of arrays is that, in Julia, they are passed by reference. This
means that two arrays that have been made equal point out to the same data in memory and
that changing one array changes the other as well.

In [3]:
a = ["My string" 2; 3.1 true]
b = a
a[1,1] = "Example of passing by reference"
b[1, 1]

"Example of passing by reference"



This can be easily checked by the typing

In [4]:
pointer_from_objref(a)

Ptr{Nothing} @0x00007cd3b44c29a0

In [5]:
pointer_from_objref(b)

Ptr{Nothing} @0x00007cd3b44c29a0

and observing that both memory addresses are the same.

# Vectors
The definition of vectors in Julia is straightforward

In [ ]:
a = [1, 2, 3] # vector

In [ ]:
a = [1; 2; 3] # same vector

Both instructions create an array{Int64, 1} , or its alias Vector{Int64} . However, you
must note that:

In [ ]:
b = [1 2 3] # 1x3 matrix (i.e., row vector)
b = [1 2 3]' # 3x1 matrix (i.e., column vector)

generate an 1 × 3 array{Int64, 2} (i.e., 1 × 3 matrix) and an 3 × 1 array{Int64, 2}
(i.e., 3 × 1 matrix), or its alias Matrix{Int64} . Therefore:

In [11]:
a = [1, 2, 3]
b = [1 2 3]
a == b

false

returns false , as we are comparing a flat array with a 1 × 3 array{Int64, 2} . Similarly,
a vector and a n × 1 matrix (i.e., a column vector) are different objects as well. Having both
vectors and matrices helps with the implementation of some operations in linear algebra.
In many applications, you might then prefer to use matrices even when dealing with onedimensional objects to avoid complications of mixing vectors and matrices. Most of operators
of manipulation of vectors below will apply to matrices without problems. But in other
applications you may want to be careful separating vectors from matrices.

A faster command to created vectors is collect() :

In [13]:
a = collect(1.0:0.5:4) # vector from 1.0 to 4.0 with step 0.5

7-element Vector{Float64}:
 1.0
 1.5
 2.0
 2.5
 3.0
 3.5
 4.0

Similarly, Julia has step range constructors

In [26]:
i = 1
n = 10
j = 2
k = 10
a = i:j:n # list of points from i to n with step j


1:2:9

In [25]:
a = range(1, 5, length=k) # linearly spaced list of k points

1.0:0.4444444444444444:5.0

that generate lists of points that are not vectors. You can always transform them back into
vectors with collect() or the ellipse:

In [27]:
i = 1
n = 10
j = 2
a = i:j:n # a list of points
b = collect(a) # creates a vector


5-element Vector{Int64}:
 1
 3
 5
 7
 9

The basic operators to manipulate vectors include:

show(a) # shows a

sum(a) # sum of a

maximum(a) # max of a

minimum(a) # min of a

a[end] # gets last element of a

a[end-1] # gets element of a -1

Also, we can sort them:18

a = [2,1,3]

sort(a) # sorts a

sort(a,by=abs) # sorts a by absolute values

sortperm(a) # indices of sort of a

find the start and end

first(a) # returns 2

last(a) # returns 3

or any arbitrary elements in them:

a = [2,1,3]

first(a) # returns 2

last(a) # returns 3

findall(isodd,a) # returns indices of occurrences (here 2,3)

Note that we can check in any collection, including arrays, the presence of an element
with the short yet powerful function in :

a = [1,2,3]

2 in a # returns true

in(2,a) # same as above

4 in a # returns false

In [28]:
a = [1,2,3]

3-element Vector{Int64}:
 1
 2
 3

In [29]:
2 in a # returns true

true

In [30]:
in(2,a) # same as above

true

In [31]:
4 in a # returns false

false

# Julia convention: The use of ! at the end of a function:

The suffix means that the function is changing the operand. For example:

sort!(a) # sorts a and changes it

popfirst!(a) # eliminates first element of a

pushfirst!(a,c) # adds c as an additional element of a at its start

pop!(a) # eliminates last element of a

push!(a,c) # adds c as an additional element of a at its end

# Matrices

More concrete examples of matrix commands (most of the commands for vectors will also
apply to matrices):

a = [1 2; 3 4] # create a 2x2 matrix

a[2, 2] # access element 2,2

a[1, :] # access first row

a[:, 1] # access first column

a = zeros(2,2) # zero matrix

a = ones(2,2) # unitary matrix

a = fill(2,3,4) # fill a 3x4 matrix with 2's

a = trues(2,2) # 2x2 matrix of trues

a = falses(2,2) # 2x2 matrix of falses

a = rand(2,2) # random matrix (uniform)

a = randn(2,2) # random matrix (gaussian)

If we want to repeat a matrix to take advantage of some inner structure:

a = [1 2; 3 4] # create a 2x2 matrix

b = repeat(a, 2,3) # repeats matrix 2x3 times

Matrices (and other multidimensional arrays) are stored in column-major order (as in
BLAS and LAPACK).


# The Basic Operations with Matrices

a' # complex conjugate transpose of a

a[:] # convert matrix a to vector

vec(a) # vectorization of a

a*B # multiplication of two matrices

a\b # solution of linear system ax = b

A few more advanced operations with matrices:

inv(a) # inverse of a

rank(a) # rank of a

norm(a) # Euclidean norm of a

det(a) # determinant of a

trace(a) # trace of a

eigen(a) # eigenvalues and eigenvectors




# I/O

Julia works in streams of data for I/O. The basic printing functionality is
a = 1
print(a) # basic printing functionality, no formatting
println(a) # as before, plus a newline

The basic reading functionality is

a = readline()

To deal with files, one needs to open them with a mode of operation and get a handle

f = open("results.txt", "r") # open file "results.txt"

The modes of operation of the file are:

r read

r+ read, write

w write, create, truncate

w+ read, write, create, truncate

a write, create, append

a+ read, write, create, append

In [50]:
a = 1
print(a) # basic printing functionality, no formatting
println(a) # as before, plus a newline



11


# Functions

In the tradition of programming languages in the functional approach, Julia considers functions “first-class citizens” (i.e., an entity that can implement all the operations -which are
themselves functions- available to other entities). This means, among other things, that
Julia likes to work with functions without side effects and that you can follow the recent
boom in functional programming without jumping into purely functional language.

Recall that functions in Julia use methods with multiple dispatch: each function can
be associated with hundreds of different methods. Furthermore, you can add methods to an
already existing function.

There are two ways to create a function

In [ ]:
# One-line
myfunction1(var) = var+1

# Several lines
function myfunction2(var1, var2="Float64", var3=1)
  output1 = var1+2
  output2 = var2+4
  output3 = var3+3 # var3 is optional, by default var3=1
  return [output1 output2 output3]
end

Note that tab indentation is not required by Julia; we only introduce it for visual appeal. In
the second function, var2 = ”F loat64” fixed the type of the second argument and var3 = 1
pins a default value for the third argument, which becomes optional. We can also have
keyword argument, which can be ommitted

In [ ]:
function myfunction3(var1, var2; keyword=2)
  output1 = var1+var2+keyword
end

The difference between an optional argument and a keyword is that the keyword can appear
in any place of the function call while the optional argument must appear in order

To have several methods associated to a function, you only need to specify the type of
the operands:

In [33]:
function myfunction3(var1::Int64, var2; keyword=2)
  output1 = var1+var2+keyword
end
function myfunction3(var1::Float64, var2; keyword=2)
  output1 = var1/var2+keyword
end
myfunction3(2,1) # returns 5


5

In [34]:
myfunction3(2.0,1) # returns 4.0

4.0

# Recursion

Recursion is a function that calls itself:

In [35]:
fib(n) = n < 2 ? n : fib(n-1) + fib(n-2)

fib (generic function with 1 method)

In [37]:
fib(6)

8

# MapReduce

Julia supports generic function applicators. First, we have map() :

In [51]:
map(floor,[1.2, 5.6, 2.3]) # applies floor to vector [1.2, 5.6, 2.3]
map(x ->x^2,[1.2, 5.6, 2.3]) # applies abstract to vector [1.2, 5.6, 2.3]

3-element Vector{Float64}:
  1.44
 31.359999999999996
  5.289999999999999

map() also works for multiple inputs:

In [52]:
map((x,y) ->x+2*y,[1,2], [3,4])

2-element Vector{Int64}:
  7
 10

An alternative syntax is with do-end

In [53]:
map([1.2, 5.6, 2.3]) do x
  floor(x)
end

3-element Vector{Float64}:
 1.0
 5.0
 2.0

Second, we have reduce() and associated folding functions

In [54]:
reduce(+,[1,2,3]) # generic reduce
foldl(-,[1,2,3]) # folding (reduce) from the left
foldr(-,[1,2,3]) # folding (reduce) from the right

2

Third, we can directly apply mapreduce()

In [55]:
mapreduce(x->x^2, +, [1,3])

10

Finally, we have the related function filter()

In [56]:
a = [1,5,8,10,12]
filter(isodd,a) # select odd elements of a

2-element Vector{Int64}:
 1
 5

# Loops

Julia provides with basic loops, including breaks and continues:

In [57]:
# basic loop
a = [1, 2, 3]
for i in a
# do something
end
# loop with a break
a = [1, 2, 3]
for i in a
# do something until a condition is satisfied
break
end
# loop with a continue
a = [1, 2, 3]
for i in a
# jump to next step of the iteration if a condition is satisfied
continue
end

Loops can be used to define arrays in comprehensions (a ruled-defined array)

In [58]:
[n^2 for n in 1:5] # basic comprehensions
Float64[n^2 for n in 1:5] # comprehension fixing type

5-element Vector{Float64}:
  1.0
  4.0
  9.0
 16.0
 25.0

Julia complements standard loops with comprehensions and whiles

In [60]:
# Comprehensions
[exp(i) for i in 1:5]
# basic while
N = 5
i = 1
while i <= N
# do something
  i = i + 1
end

# Conditionals

Julia has both traditional if-then statements

In [65]:
if i < N
    # do something
    k = 0
  elseif i > N
    # do something else
    k = 1
  else
    # do something even more different
    k = 2
end

1

and efficient ternary expressions condition ? do something : do something else such
as

In [67]:
a = 2
a<2 ? b = 1 : b = 2

2

# Tuples and Dictionaries

Tuples is a data type of that contains an ordered collection of elements. The elements of a
tuple cannot be changed once they have been defined

In [68]:
a = ("This is a tuple", 2018) # definition of a tuple
a[2] # accessing element 2 of tuple a

2018

We can create tuples with zip

In [69]:
a = [1 2]
b = [3 4]
c = zip(a,b)
print(c)

zip([1 2], [3 4])

Dictionaries are associative collections with keys (names of elements) are values of elements

In [70]:
# Creating a dictionary
a = Dict("University of Pennsylvania" => "Philadelphia", "Boston College" =>
"Boston")
a["University of Pennsylvania"] # access one key
a["Harvard"] = "Cambridge" # adds an additional key
delete!(a,"Harvard") # deletes a key
keys(a)
values(a)
haskey(a,"University of Pennsylvania") # returns true
haskey(a,"MIT") # returns false

false

# A Julia Package to program NVIDIA GPUs: CUDA

In [ ]:
import Pkg; Pkg.add("CUDA")
import Pkg; Pkg.add("BenchmarkTools")
using BenchmarkTools
try
    using CUDA
catch
    println("No GPU found.")
else
    run(`nvidia-smi`)
    # Create a new random matrix directly on the GPU:
    M_on_gpu = CUDA.CURAND.rand(2^11, 2^11)
    @btime $M_on_gpu * $M_on_gpu; nothing
end

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
   Installed BenchmarkTools ─ v1.6.0
    Updating `~/.julia/environments/v1.10/Project.toml`
  [6e4b80f9] + BenchmarkTools v1.6.0
    Updating `~/.julia/environments/v1.10/Manifest.toml`
  [6e4b80f9] + BenchmarkTools v1.6.0
  [9abbd945] + Profile
Precompiling packages...
   1976.1 ms  ✓ BenchmarkTools
  1 dependency successfully precompiled in 3 seconds. 460 already precompiled.


Fri Apr  4 16:37:36 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Need Help?

* Learning: https://julialang.org/learning/
* Documentation: https://docs.julialang.org/
* Questions & Discussions:
  * https://discourse.julialang.org/
  * http://julialang.slack.com/
  * https://stackoverflow.com/questions/tagged/julia

If you ever ask for help or file an issue about Julia, you should generally provide the output of `versioninfo()`.

Add new code cells by clicking the `+ Code` button (or _Insert_ > _Code cell_).

Have fun!

<img src="https://raw.githubusercontent.com/JuliaLang/julia-logo-graphics/master/images/julia-logo-mask.png" height="100" />